In [1]:
%load_ext autoreload
%autoreload 2

import ablang

In [2]:
model = ablang.pretrained("/vols/bitbucket/olsen/models/ABLANG-709/")

In [34]:
seq1 = {
    'H':'EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    'L':'DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
}
seq2 = {
    'H':'EVQLLGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    'L':'SISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
}
seq3 = {
    'H':'EVQLLGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    }
seq4 = {
    'H':'EVQLQESGGGLVQPGESLRLSCVGSGSSFGESTLSYYAVSWVRQAPGKGLEWLSIINAGGGDIDYADSVEGRFTISRDNSKETLYLQMTNLRVEDTGVYYCAKHMSMQQVVSAGWERADLVGDAFDVWGQGTMVTVSS',
    'L':'DIQLTQSPSSLSASVGDRVTLTCQASQDIRKFLNWYQQKPGKGPKLLIYDASNLQRGVPSRFSGGGSGTDFTLIISSLQPEDVGTYYCQQYDGLPFTFGGGTKVVIK'
}

seqs = [seq1, seq2]

In [58]:
seqs_new = [
    {'H': 'QVQLQESGPRLVKPSETLSLTCTVSGGSISGSNYFWGWIRQPPGKGLEWIGTIYYGGSTYYNPSLKSRVTISVDTSENQFSLKLSSVTAADTAVYYCANLAWLRGYFDYWGQGTLVTVSS',
  'L': 'NFMLTQPHSVSESPGKTVTISCTRSSGSIASNYVQWYQQRPGSAPTTVIYEDNQRPSGVPDRFSGSIDSSSNSVSLTISGLKTEDEADYYCQSYDSSNWVFGGGTKLTVL'},
    {'H': 'EVQLVQSGAEVKKPGASVKVSCKASGYTFTDYWIYWVRQAPGQGLEWMGTIDTSDHKTSYNQKFRGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCARDFIITVNYPVVDYWGQGTLVTVSS',
  'L': 'QIVLTQSPATLSLSPGERATLSCSAGSSVNYMHWFQQKPGQAPRLLIYSTSNLASGVPARFSGSGSGTDYTLTISSLEPEDFAVYYCQQRSSLPFTFGQGTKLEIK'},
    
    ]

In [71]:
results = model(seqs_new, mode='rescoding', align=True, chunk_size=2)
results

aligned_results(aligned_embeds=array([[[-0.3878253400325775, -0.48439493775367737,
         -0.06649225205183029, ..., 0.042517926543951035,
         -0.4628494679927826, '<'],
        [-0.4599864184856415, 0.4597696363925934, -0.1467791050672531,
         ..., 0.0333760529756546, -0.45324841141700745, 'E'],
        [-0.1835758090019226, 0.3269803524017334, -0.16753999888896942,
         ..., -0.21250931918621063, 0.0888785868883133, 'V'],
        ...,
        [-0.29478898644447327, -0.061758678406476974,
         -0.3153388500213623, ..., -0.5813659429550171,
         0.23931853473186493, 'I'],
        [-0.029618775472044945, 0.3093995749950409,
         -0.21040979027748108, ..., -0.4987454116344452,
         0.24815092980861664, 'K'],
        [-0.15065643191337585, -0.3086070418357849, -0.1835339218378067,
         ..., 0.24800609052181244, -0.4314933121204376, '>']],

       [[-0.38041818141937256, -0.3612355589866638,
         0.037833377718925476, ..., 0.10084830969572067,
      

In [72]:
results.aligned_embeds.shape

(2, 240, 481)

In [73]:
results.number_alignment.shape

(240,)